In [1]:
import os

os.environ["TAVILY_API_KEY"] = "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"


In [2]:
import os

os.environ["NOMIC_API_KEY"] = "nk-s1262hYIqiiWAPWQehKgEswm0sdiZqw-Qa2zt6KdOEs"


In [1]:
import requests

def fetch_readme_content(repo_url):
    # Construct the raw URL for the README.md file
    if repo_url.endswith('/'):
        repo_url = repo_url[:-1]
    raw_url = repo_url.replace('github.com', 'raw.githubusercontent.com') + '/main/README.md'
    
    # Fetch the content of the README.md file
    response = requests.get(raw_url)
    if response.status_code == 200:
        return response.text
    else:
        raise FileNotFoundError(f"README.md not found at {raw_url}")


In [2]:
from langchain.text_splitter import CharacterTextSplitter

def chunk_readme_content(readme_content):
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    return text_splitter.split_text(readme_content)


In [10]:
from langchain_nomic import NomicEmbeddings
from langchain.vectorstores import FAISS

def create_vector_store(chunks):
    embeddings = NomicEmbeddings(model="nomic-embed-text-v1.5")
    vector_store = FAISS.from_texts(chunks, embeddings)
    return vector_store


In [7]:
from langchain.llms import Ollama
def setup_llm():
    llm = Ollama(model="llama3.2:latest", temperature=0)
    return llm


In [5]:
from langchain.chains import RetrievalQA

def create_retrieval_qa_chain(vector_store, llm):
    retriever = vector_store.as_retriever()
    qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff")
    return qa_chain


In [8]:
def summarize_readme(repo_url):
    # Fetch the README.md content
    readme_content = fetch_readme_content(repo_url)
    
    # Chunk the content
    chunks = chunk_readme_content(readme_content)
    
    # Create a vector store
    vector_store = create_vector_store(chunks)
    
    # Set up the Llama LLM
    llm = setup_llm()
    
    # Create a RetrievalQA chain
    qa_chain = create_retrieval_qa_chain(vector_store, llm)
    
    # Generate the summary
    summary = qa_chain.run("Provide a summary of this README.md file.")
    return summary


In [12]:
repo_url = "https://github.com/catamaican/AED"
summary = summarize_readme(repo_url)
print(summary)


I don't know what the question is. The provided context appears to be a snippet from a README.md file, but it doesn't specify what the question is or what information is being requested.
